## Data cleaning and preparation

`data.tsv` downloaded from the official IMDB Developer website (https://developer.imdb.com/non-commercial-datasets/). It contains over 10,000,000 rows of data, each row representing a title, including movies, TV-shows, TV-Show episodes and videogames, and more.

In this notebook, I will clean the data and remove unnecessary columns, preparing it for insertion into the database.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.tsv', sep='\t')
df.head()

/var/folders/j4/l52yqm311px4czkrv1_6txsc0000gn/T/ipykernel_22367/2428849113.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.tsv', sep='\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [2]:
print("Titles in dataset: {:,}".format(df.shape[0]))

Titles in dataset: 10,384,671


Getting unique values from the `titleType` and `genres` columns, to see what kind of titles are included in the dataset. This will be useful in testing the API as well as implementing the drop-down menus in the frontend.

In [3]:
df.titleType.unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [4]:
df['genres'].str.split(',').explode().unique()

array(['Documentary', 'Short', 'Animation', 'Comedy', 'Romance', 'Sport',
       'News', 'Drama', 'Fantasy', 'Horror', 'Biography', 'Music', 'War',
       'Crime', 'Western', 'Family', 'Adventure', 'Action', 'History',
       'Mystery', '\\N', 'Sci-Fi', 'Musical', 'Thriller', 'Film-Noir',
       'Talk-Show', 'Game-Show', 'Reality-TV', 'Adult', nan], dtype=object)

Dropping unnecessary columns, and including only movies, TV-shows, TV movies, TV episodes, TV mini-series and TV specials:

In [5]:
df = df.drop(columns=['tconst'])
df = df.drop(columns=['endYear'])
df = df[df.titleType.isin(['movie', 'tvMovie', 'tvSeries', 'tvEpisode', 'tvMiniSeries', 'tvSpecial'])]

print("Titles after filtering: {:,}".format(df.shape[0]))

Titles after filtering: 9,088,256


Dropping rows with missing/incorrect values:

In [6]:
# finding out how missing values are represented in integer columns
nan_runtime_minutes = df[~df.runtimeMinutes.str.isnumeric()].runtimeMinutes.unique()
nan_start_years = df[~df.startYear.str.isnumeric()].startYear.unique()

print("Unique non-numeric values in runtimeMinutes: {}".format(nan_runtime_minutes))
print("Unique non-numeric values in startYear: {}".format(nan_start_years))

Unique non-numeric values in runtimeMinutes: ['\\N' 'Reality-TV' 'Talk-Show' 'Documentary' 'Game-Show,Short'
 'Animation,Comedy,Family' 'News,Talk-Show' 'Comedy,News,Talk-Show'
 'Documentary,Reality-TV' 'Adult' 'Game-Show,Reality-TV']
Unique non-numeric values in startYear: ['\\N']


In [7]:
# removing all rows where runtimeMinutes is not numeric
df = df[df.runtimeMinutes.str.isnumeric()]
print("Titles after dropping non-numeric runtimeMinutes: {:,}".format(df.shape[0]))

Titles after dropping non-numeric runtimeMinutes: 2,298,462


In [8]:
# dealing with the rest, \\N is used to represent missing values in this dataset
df = df.replace('\\N', np.nan).dropna()
print("Titles after dropping missing values: {:,}".format(df.shape[0]))

Titles after dropping missing values: 2,037,287


In [9]:
df = df.reset_index(drop=True)
df.head()

,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,movie,Miss Jerry,Miss Jerry,0,1894,45,Romance
1,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,100,"Documentary,News,Sport"
2,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography"
3,movie,The Prodigal Son,L'enfant prodigue,0,1907,90,Drama
4,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy"


Saving `.csv` to be used in a script to populate the database:

In [11]:
df.to_csv('data.csv', index=False)